In [1]:
import os
import ee
import numpy as np
import rsfuncs as rs
import pandas as pd
import geopandas as gp

import matplotlib.pyplot as plt

from tqdm import tqdm
from osgeo import gdal
from shapely.geometry import Polygon

ee.Initialize()

In [2]:
# Lookup table with product names and EE ids 

prod_lookup = {
    'disal' : ee.ImageCollection("users/aguzman/Kaweah_disalexi"),
    'metric' : ee.ImageCollection("users/aguzman/Kaweah_eemetric"),
    'ptjpl' : ee.ImageCollection("users/aguzman/Kaweah_ptjpl"),
    'sebal': ee.ImageCollection("users/aguzman/Kaweah_sebal"),
    'sims' : ee.ImageCollection("users/aguzman/Kaweah_sims"),
    'ssebop' : ee.ImageCollection("users/aguzman/Kaweah_ssebop")
}

In [3]:
# Helpers 

def dict2arr(data_dict, var_name):
    '''converts ee dictionary output from .getInfo() to a numpy array. Wraps array_from_df'''
    
    data = data_dict[var_name]
    lats = data_dict['latitude']
    lons = data_dict['longitude']

    df = pd.DataFrame([data,lats,lons]).T
    df.columns = [var_name, "latitude", 'longitude']
    arr = rs.array_from_df(df, var_name)
    
    return arr


# Main routine

def process_imcol(model_name, n_timestep):
        
    # Set some params
    num_images = months
    model_name = model_name
    model_dir = os.path.join("../rasters/openet",model_name)
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    print("{}  =  WRITE DIR".format(model_dir))

    # Loop through polygons 
    for polyidx, poly in tqdm(polycoords.items()):
        
        print("Downloading data")

        # get aoi as gdf
        aoi = ee.Geometry.Polygon(poly)
        coords = aoi.getInfo()['coordinates'][0]
        lons = []
        lats = []

        for ll in coords:
            lon,lat = ll
            lats.append(lat)
            lons.append(lon)

        polygon_geom = Polygon(zip(lons, lats))
        crs = 4326
        aoi_gdf = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])   

        # Loop through time slices
        for im_idx in range(0,num_images):

            # setup time slice dir (hope this is good idea)
            timedir = os.path.join(model_dir,"time_{}".format(im_idx))
            if not os.path.exists(timedir):
                os.mkdir(timedir)

            resultfn = "{}_p{}_t{}.tif".format(model_name,str(polyidx), str(im_idx))
            outfn = os.path.join(timedir,resultfn)

            # write out if the outfn doesn't yet exist
            if not os.path.exists(outfn):
                imdat = ee.Image.pixelLonLat().addBands(imList.get(im_idx)).reduceRegion(reducer=ee.Reducer.toList(),geometry=aoi,scale=30)
                et_im = dict2arr(imdat.getInfo(), 'et')
                rs.write_raster(et_im, aoi_gdf,outfn)        
    
    # After rasters are written:
    # Loop through the time dirs, 
    # gdal_merge the tiffs, 
    # write the merge in ../rasters/openet/merged_tn.tif where n is time index
    for im_idx in range(0,num_images):

        timedir = os.path.join(model_dir,"time_{}".format(im_idx))
        merged_dir = os.path.join(model_dir,"merged")

        if not os.path.exists(merged_dir):
            os.mkdir(merged_dir)
        outfn = os.path.join(merged_dir, model_name + "_t{}.tif".format(im_idx))

        # merge and write 
        dirfiles = [os.path.join(timedir,x) for x in os.listdir(timedir) if x.endswith(".tif")]
        g = gdal.Warp(outfn, dirfiles, format="GTiff") # if you want
        g = None # Close file
    
    return "Completed {}".format(prod)
    


In [4]:
# Read the shapefile, split it
gdf = gp.read_file("../shape/kb_rpj.shp").simplify(0.09)
dx,dy = 0.25, 0.25
area = rs.gdf_to_ee_poly(gdf)

print("Splitting Geometry")
polys = rs.gen_polys(area, dx = dx, dy = dy)
polydict = polys.getInfo()

# Ensure each poly is a single geometry (should already be the case, but make sure)
valid_polys = []

for i in tqdm(polydict['features'][:]):
    aoi = ee.Geometry.Polygon(i['geometry']['coordinates']).intersection(area)
    pols = aoi.getInfo()['coordinates']
    if len(pols) == 0:
        continue
    if len(pols) == 1:
        valid_polys.append(ee.Geometry.Polygon(pols))
    if len(pols) > 1:
        for i in pols:
            valid_polys.append(ee.Geometry.Polygon(i))

# Make a dict with the poly IDs
polycoords = {}

for idx,poly in enumerate(valid_polys):
    polycoords[idx] = poly['coordinates']

Splitting Geometry


100%|██████████| 7/7 [00:00<00:00,  7.84it/s]


In [5]:
for prod, imcol in prod_lookup.items():
    print("PROCESSING {}".format(prod))
    imList = imcol.toList(imcol.size())
    months = int(imList.size().getInfo())
    
    process_imcol(prod,months)

PROCESSING + disal


  0%|          | 0/7 [00:00<?, ?it/s]

../rasters/openet/disal  =  WRITE DIR


100%|██████████| 7/7 [00:01<00:00,  6.17it/s]


PROCESSING + metric


...

wrote ../rasters/openet/ssebop/time_59/ssebop_p4_t59.tif
wrote ../rasters/openet/ssebop/time_0/ssebop_p5_t0.tif
wrote ../rasters/openet/ssebop/time_1/ssebop_p5_t1.tif
wrote ../rasters/openet/ssebop/time_2/ssebop_p5_t2.tif
wrote ../rasters/openet/ssebop/time_3/ssebop_p5_t3.tif
wrote ../rasters/openet/ssebop/time_4/ssebop_p5_t4.tif
wrote ../rasters/openet/ssebop/time_5/ssebop_p5_t5.tif
wrote ../rasters/openet/ssebop/time_6/ssebop_p5_t6.tif
wrote ../rasters/openet/ssebop/time_7/ssebop_p5_t7.tif
wrote ../rasters/openet/ssebop/time_8/ssebop_p5_t8.tif
wrote ../rasters/openet/ssebop/time_9/ssebop_p5_t9.tif
wrote ../rasters/openet/ssebop/time_10/ssebop_p5_t10.tif
wrote ../rasters/openet/ssebop/time_11/ssebop_p5_t11.tif
wrote ../rasters/openet/ssebop/time_12/ssebop_p5_t12.tif
wrote ../rasters/openet/ssebop/time_13/ssebop_p5_t13.tif
wrote ../rasters/openet/ssebop/time_14/ssebop_p5_t14.tif
wrote ../rasters/openet/ssebop/time_15/ssebop_p5_t15.tif
wrote ../rasters/openet/ssebop/time_16/ssebop_p5_t1

 86%|████████▌ | 6/7 [2:34:58<29:49, 1789.42s/it]

wrote ../rasters/openet/ssebop/time_59/ssebop_p5_t59.tif
wrote ../rasters/openet/ssebop/time_0/ssebop_p6_t0.tif
wrote ../rasters/openet/ssebop/time_1/ssebop_p6_t1.tif
wrote ../rasters/openet/ssebop/time_2/ssebop_p6_t2.tif
wrote ../rasters/openet/ssebop/time_3/ssebop_p6_t3.tif
wrote ../rasters/openet/ssebop/time_4/ssebop_p6_t4.tif
wrote ../rasters/openet/ssebop/time_5/ssebop_p6_t5.tif
wrote ../rasters/openet/ssebop/time_6/ssebop_p6_t6.tif
wrote ../rasters/openet/ssebop/time_7/ssebop_p6_t7.tif
wrote ../rasters/openet/ssebop/time_8/ssebop_p6_t8.tif
wrote ../rasters/openet/ssebop/time_9/ssebop_p6_t9.tif
wrote ../rasters/openet/ssebop/time_10/ssebop_p6_t10.tif
wrote ../rasters/openet/ssebop/time_11/ssebop_p6_t11.tif
wrote ../rasters/openet/ssebop/time_12/ssebop_p6_t12.tif
wrote ../rasters/openet/ssebop/time_13/ssebop_p6_t13.tif
wrote ../rasters/openet/ssebop/time_14/ssebop_p6_t14.tif
wrote ../rasters/openet/ssebop/time_15/ssebop_p6_t15.tif
wrote ../rasters/openet/ssebop/time_16/ssebop_p6_t1

100%|██████████| 7/7 [3:14:39<00:00, 1668.51s/it]

wrote ../rasters/openet/ssebop/time_59/ssebop_p6_t59.tif
